In [ ]:
import numpy as np
import pandas as pd
import json
import tensorflow as tf
import os
from keras.preprocessing import image

df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
[os.mkdir(os.path.join('/kaggle/working', "label_" + str(x))) for x in df.label.unique()]

from shutil import copyfile

for a,b in df.iterrows():
    src = os.path.join('/kaggle/input/cassava-leaf-disease-classification/train_images',b.image_id)
    dst = os.path.join('/kaggle/working', "label_" + str(b.label), b.image_id)
    copyfile(src, dst)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
import os
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers

train_data_dir = '/kaggle/working/'
img_height = 150
img_width = 150
batch_size = 32
num_classes = 5

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.3,
    rotation_range=40,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator, epochs=1, validation_data = validation_generator, verbose = 1)

In [ ]:
model.save('standard.model')

In [ ]:
test_path = '../input/cassava-leaf-disease-classification/test_images/'
test_csv = '../input/cassava-leaf-disease-classification/sample_submission.csv'
submission = pd.read_csv(test_csv)

test_images = []
test_image_names = []

for img in submission['image_id']:
    test_image_names.append(img)
    img = image.load_img(os.path.join(test_path, img), target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    test_images.append(x)

#for i in os.listdir(test_path):
#    test_image_names.append(i)
#    img = image.load_img(os.path.join(test_path, i), target_size=(150,150))
#    x = image.img_to_array(img)
#    x = np.expand_dims(x, axis=0)
#    test_images.append(x)

#images = np.vstack([test_images])
classes = model.predict_classes(test_images, batch_size=10)
print(classes)

In [ ]:
#sub = pd.DataFrame()
#sub = pd.concat([pd.Series(test_image_names), pd.Series(classes)], axis=1) 
#sub.columns = ['image_id', 'label']
submission['label'] = classes
submission.to_csv('submission.csv', index = False)
submission.to_csv('/kaggle/working/submission.csv', index = False)